In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_excel('data/sample - default of credit card clients.xls', skiprows=1, index_col='ID')
target_column = 'default payment next month'
features = df.drop(target_column, axis=1)
targets = df[target_column]
train_x, test_x, train_y, test_y = train_test_split(features, targets, stratify=targets, random_state=42)
scaler = StandardScaler()
scaled_train_x = scaler.fit_transform(train_x)
scaled_test_x = scaler.transform(test_x)
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
0,30000,1,2,2,25,0,0,0,0,0,...,12580,13716,14828,1500,2000,1500,1500,1500,2000,0
1,150000,2,1,2,26,0,0,0,0,0,...,101581,77741,77264,4486,4235,3161,2647,2669,2669,0
2,70000,2,3,1,32,0,0,0,0,0,...,69753,70111,70212,2431,3112,3000,2438,2500,2554,0
3,130000,1,3,2,49,0,0,0,0,0,...,16898,11236,6944,1610,1808,7014,27,7011,4408,0
4,50000,2,2,2,36,0,0,0,0,0,...,19574,20295,19439,2000,1500,1000,1800,0,1000,1


In [2]:
from sklearn.svm import LinearSVC

lsvc = LinearSVC()
lsvc.fit(scaled_train_x, train_y)
print(lsvc.score(scaled_train_x, train_y))
print(lsvc.score(scaled_test_x, test_y))

0.7971555555555555
0.7877333333333333


C:\Users\words\anaconda3\envs\datasci\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [3]:
targets.value_counts(normalize=True)

0    0.783067
1    0.216933
Name: default payment next month, dtype: float64

In [4]:
from sklearn.svm import SVC

svc = SVC(probability=True)
svc.fit(scaled_train_x, train_y)
print(svc.score(scaled_train_x, train_y))
print(svc.score(scaled_test_x, test_y))

0.8272
0.8181333333333334


In [5]:
svc.predict_proba(scaled_test_x)

array([[0.8569643 , 0.1430357 ],
       [0.41377586, 0.58622414],
       [0.18842363, 0.81157637],
       ...,
       [0.83985279, 0.16014721],
       [0.88162874, 0.11837126],
       [0.84109799, 0.15890201]])

In [6]:
from pycaret.distributions import UniformDistribution
from pycaret.classification import setup, create_model, tune_model

clf_setup = setup(data=df, target='default payment next month', normalize=True)
lsvc = create_model(LinearSVC())

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8190,0.0000,0.3125,0.6604,0.4242,0.3328,0.3657
1,0.8133,0.0000,0.2857,0.6400,0.3951,0.3033,0.3379
2,0.8286,0.0000,0.3482,0.6964,0.4643,0.3755,0.4075
3,0.8343,0.0000,0.4286,0.6761,0.5246,0.4303,0.4467
4,0.8267,0.0000,0.4286,0.6400,0.5134,0.4129,0.4252
5,0.8057,0.0000,0.2679,0.6000,0.3704,0.2749,0.3062
6,0.8076,0.0000,0.2857,0.6038,0.3879,0.2907,0.3194
7,0.8229,0.0000,0.3393,0.6667,0.4497,0.3572,0.3862
8,0.8229,0.0000,0.3304,0.6727,0.4431,0.3521,0.3836
9,0.8321,0.0000,0.3214,0.7500,0.4500,0.3691,0.4154


In [7]:
tuned_lsvc = tune_model(lsvc, search_library='scikit-optimize', custom_grid={"C": UniformDistribution(0, 50)})

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8038,0.0000,0.2857,0.5818,0.3832,0.2824,0.3077
1,0.8114,0.0000,0.3036,0.6182,0.4072,0.3103,0.3381
2,0.8248,0.0000,0.3393,0.6786,0.4524,0.3616,0.3924
3,0.8229,0.0000,0.4107,0.6301,0.4973,0.3955,0.4089
4,0.8152,0.0000,0.3929,0.6027,0.4757,0.3695,0.3820
5,0.8133,0.0000,0.2946,0.6346,0.4024,0.3090,0.3410
6,0.8038,0.0000,0.2589,0.5918,0.3602,0.2648,0.2964
7,0.8190,0.0000,0.3214,0.6545,0.4311,0.3381,0.3684
8,0.8210,0.0000,0.3214,0.6667,0.4337,0.3425,0.3747
9,0.8302,0.0000,0.2857,0.7805,0.4183,0.3430,0.4028


In [8]:
tuned_lsvc

LinearSVC(C=16.447780595007984, class_weight=None, dual=True,
          fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
          max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
          tol=0.0001, verbose=0)

In [9]:
lsvc = create_model('svm')
tuned_lsvc = tune_model(lsvc, search_library='scikit-optimize')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8190,0.0000,0.2679,0.6977,0.3871,0.3048,0.3531
1,0.8248,0.0000,0.2679,0.7500,0.3947,0.3182,0.3762
2,0.8343,0.0000,0.3125,0.7778,0.4459,0.3687,0.4219
3,0.8305,0.0000,0.3571,0.7018,0.4734,0.3848,0.4161
4,0.8286,0.0000,0.3839,0.6719,0.4886,0.3947,0.4171
5,0.8057,0.0000,0.2411,0.6136,0.3462,0.2567,0.2955
6,0.8095,0.0000,0.2411,0.6429,0.3506,0.2651,0.3092
7,0.8076,0.0000,0.2411,0.6279,0.3484,0.2609,0.3023
8,0.8286,0.0000,0.3036,0.7391,0.4304,0.3496,0.3977
9,0.8263,0.0000,0.2768,0.7561,0.4052,0.3283,0.3855


In [10]:
lsvc

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.001, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=590, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [11]:
rbfsvc = create_model('rbfsvm')
tuned_rfbsvc = tune_model(rbfsvc, search_library='scikit-optimize')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8152,0.7140,0.2679,0.6667,0.3822,0.2961,0.3388
1,0.8210,0.6898,0.2768,0.7045,0.3974,0.3150,0.3627
2,0.8286,0.7346,0.3214,0.7200,0.4444,0.3602,0.4013
3,0.8381,0.7574,0.3929,0.7213,0.5087,0.4217,0.4496
4,0.8305,0.7599,0.4018,0.6716,0.5028,0.4083,0.4279
5,0.8076,0.6765,0.2500,0.6222,0.3567,0.2671,0.3056
6,0.8038,0.6978,0.2232,0.6098,0.3268,0.2399,0.2816
7,0.8171,0.7106,0.2768,0.6739,0.3924,0.3062,0.3484
8,0.8229,0.7345,0.2946,0.7021,0.4151,0.3307,0.3741
9,0.8302,0.7448,0.3036,0.7556,0.4331,0.3540,0.4051
